In [ ]:
import pvl
import struct
import matplotlib.pyplot as plt
import numpy as np
import datetime
import os.path
import binascii

In [ ]:
mimap_file = '/Users/kdlee/builds/ISIS3/isis/tests/data/mimap2isis/MI_MAP_02_N65E328N64E329SC.img'
image_file = mimap_file

In [ ]:
header = pvl.load(mimap_file)
header

In [ ]:
with open(mimap_file, 'rb') as f:
    try:
        image_offset = header["^IMAGE"].value-(header['IMAGE']['SAMPLE_BITS']//8)
        f.seek(image_offset)
        b_image_data = f.read()
    except AttributeError:
        # If detached label, "^IMAGE" will be a list.
        image_file = os.path.dirname(mimap_file) + "/" + header["^IMAGE"][0]
        image_offset = header["^IMAGE"][1].value
        with open(image_file, 'rb') as im_f:
            b_image_data = im_f.read()

In [ ]:
n_lines = 5
n_samples = 5
line_length = header['IMAGE']['LINE_SAMPLES'] * (header['IMAGE']['SAMPLE_BITS']//8)

In [ ]:
image_data = []
# For MI_L3C: Use a good subset of the data.
# Starting at the top left corner resulted in all NULL pixels.
line_offset = 40
sample_offset = 3000
for j in range(line_offset, n_lines * header['IMAGE']['BANDS'] + line_offset + 1):
    image_sample = np.frombuffer(b_image_data[j*line_length:(j+1)*line_length], dtype=np.int16, count=int(line_length/2))
    image_data.append(image_sample)
image_data = np.array(image_data)
image_data = image_data[:, sample_offset:sample_offset + n_samples]

In [ ]:
plt.imshow(image_data)

In [ ]:
image_fn, image_ext = os.path.splitext(image_file)
crop = '_cropped'
mini_image_fn = image_fn + crop + image_ext
mini_image_bn = os.path.basename(mini_image_fn)

grammar = pvl.grammar.ISISGrammar()
grammar.comments+=(("#", "\n"), )
encoder = pvl.encoder.ISISEncoder()

# Overwrite the number of lines and samples in the label
header['IMAGE']['LINES'] = n_lines
header['IMAGE']['LINE_SAMPLES'] = n_samples

if mimap_file != image_file:
    # If detached label, point the mini label to the mini image
    header['^IMAGE'] = [mini_image_bn, pvl.collections.Units(1, 'BYTES')]
    header['FILE_NAME'] = mini_image_bn
else:
    # If attached label, calculate the new offset
    header['^IMAGE'] = pvl.collections.Units(len(pvl.dumps(header, encoder=encoder, grammar=grammar)), 'BYTES')

In [ ]:
label_fn, label_ext = os.path.splitext(mimap_file)
out_label = label_fn + crop + label_ext

pvl.dump(header, out_label, encoder=encoder, grammar=grammar)

In [ ]:
with open(mini_image_fn, 'ab+') as f:
    b_reduced_image_data = image_data.tobytes()
    f.seek(0, 2)
    f.write(b'\n')
    f.write(b_reduced_image_data)